# Deploy to an batch endpoint

Imagine a health clinic takes patient measurements all day, saving the details for each patient in a separate file. Then overnight, the **OJ Sales Dominicks Store 4128 forecasting**  model can be used to process all of the future forecasting as a batch, generating predictions that will be waiting the following morning so that the clinic can follow up with Dominicks stores data that are predicted to be at @ certain sales. With Azure Machine Learning, you can accomplish this by creating a batch endpoint; and that's what you'll implement in this exercise.

## Before you start

You'll need the latest version of the  **azure-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [3]:
pip install azure-ai-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 66.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.6/290.6 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.9/155.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.2.2
    Uninstalling psutil-

## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [4]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [5]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Register the model

Batch deployments can only deploy models registered in the workspace. You'll register an MLflow model, which is stored in the local `model` folder. 

In [6]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci00b7cf28ae7f478cbf/code/Users/User1-52618447/DP100-project/Final model/notebook'

In [8]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
folder_path_model = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci00b7cf28ae7f478cbf/code/Users/User1-52618447/DP100-project/Final model/scripts/mlruns/923865445274044016/bfa291c9fe12495d9ef7715ac2b6a421/artifacts/sarimax_model"
model_name = 'sarimax_model'
model = ml_client.models.create_or_update(
    Model(name=model_name, path=folder_path_model, type=AssetTypes.MLFLOW_MODEL)
)

Uploading sarimax_model (24.86 MBs): 100%|██████████| 24864300/24864300 [00:00<00:00, 85176454.83it/s]




## Create a batch endpoint

A batch endpoint is an HTTPS endpoint that applications can call to trigger a batch scoring job. A batch endpoint name needs to be unique within an Azure region. You'll use the `datetime` function to generate a unique name based on the current date and time. 

In [9]:
import datetime

endpoint_name = "batch-" + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint_name

'batch-06301354336591'

To create an endpoint with the `BatchEndpoint` class, you need to specify the name and optionally a description. After creating an endpoint, you'll deploy a model to the endpoint.

In [10]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for forecasting dominicks sales fore store 4128",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.


In [11]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="ojdominicks-store4128",
    description="A time series forecasting model",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/azure/ai/ml/entities/_deployment/batch_deployment.py:137: UserWarning: This class is intended as a base class and it's direct usage is deprecated. Use one of the concrete implementations instead:
* ModelBatchDeployment - For model-based batch deployments
* PipelineComponentBatchDeployment - For pipeline component-based batch deployments
  warnings.warn(


<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [12]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [14]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci00b7cf28ae7f478cbf/code/Users/User1-52618447/DP100-project/Final model/data/X_test.csv"
dataset_name = "forecasting-test-data"

forecasting_test_set = Data(
    path=data_path,
    type=AssetTypes.URI_FILE,
    description="A forecasting sales data from OJ for store 4128 dominicks",
    name=dataset_name,
)
ml_client.data.create_or_update(forecasting_test_set)

Data({'path': 'azureml://subscriptions/604342bf-c5b7-46e8-8eef-def8c102e8b6/resourcegroups/rg-dp100-l00b7cf28ae7f478cbf/workspaces/mlw-dp100-l00b7cf28ae7f478cbf/datastores/workspaceblobstore/paths/LocalUpload/7555d43b746c8321fde484982c016854/X_test.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'forecasting-test-data', 'description': 'A forecasting sales data from OJ for store 4128 dominicks', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/604342bf-c5b7-46e8-8eef-def8c102e8b6/resourceGroups/rg-dp100-l00b7cf28ae7f478cbf/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-l00b7cf28ae7f478cbf/data/forecasting-test-data/versions/2', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci00b7cf28ae7f478cbf/code/Users/User1-52618447/DP100-project/Final model/notebo

In [16]:
forecasting_test_set = ml_client.data.get(
    name="forecasting-test-data", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [17]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FILE, path=forecasting_test_set.id)

In [18]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

PipelineJob({'inputs': {}, 'outputs': {}, 'jobs': {}, 'component': PipelineComponent({'latest_version': None, 'intellectual_property': None, 'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.JOB', 'is_anonymous': True, 'auto_delete_setting': None, 'name': 'azureml_anonymous', 'description': "Attempting to create pipeline submission settings for endpoint: 'batch-06301354336591', deployment: 'ojdominicks-store4128'.", 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci00b7cf28ae7f478cbf/code/Users/User1-52618447/DP100-project/Final model/notebook', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x708de98461a0>, 'version': '1', 'schema': None, 'type': 'pipeline', 'display_name': 'patient_crayon_5vxdg91n', 'is_deterministic': None, 'inputs': {}, 'outputs': {}, 'yaml_str': None, 'other_parameter': {}, 'jobs': {}, 'job_types': {}, 'job_sources': {}, 'source_job_id': None}), 'type': 'pipeline', 'status': 'Preparing', 'log_files': None, 'name': 'batchjob-9b7f10a4-690d-43a4-8e74-22ac1c58dca2', 'description': "Attempting to create pipeline submission settings for endpoint: 'batch-06301354336591', deployment: 'ojdominicks-store4128'.", 'tags': {'outputType': 'output_data', 'output_data_name': None, 'inputType': 'input_data', 'azureml.batchrun': 'true', 'azureml.deploymentname': 'ojdominicks-store4128', 'azureml.jobtype': 'azureml.batchjob'}, 'properties': {'azureml.deploymentname': 'ojdominicks-store4128', 'azureml.endpointname': 'batch-06301354336591', 'azureml.pipelineid': 'e0952fae-1e04-4a19-9631-019bf99e6fdb', 'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'Unavailable', 'runType': 'HTTP', 'azureml.parameters': '{"run_max_try":"3","run_invocation_timeout":"300","mini_batch_size":"2","error_threshold":"-1","logging_level":"INFO","process_count_per_node":"2","NodeCount":"2","append_row_file_name":"predictions.csv"}', 'azureml.continue_on_step_failure': 'False', 'azureml.continue_on_failed_optional_input': 'False', 'azureml.pipelineComponent': 'pipelinerun'}, 'print_as_yaml': False, 'id': '/subscriptions/604342bf-c5b7-46e8-8eef-def8c102e8b6/resourceGroups/rg-dp100-l00b7cf28ae7f478cbf/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-l00b7cf28ae7f478cbf/jobs/batchjob-9b7f10a4-690d-43a4-8e74-22ac1c58dca2', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci00b7cf28ae7f478cbf/code/Users/User1-52618447/DP100-project/Final model/notebook', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x708de9847700>, 'serialize': <msrest.serialization.Serializer object at 0x708de98449d0>, 'display_name': 'patient_crayon_5vxdg91n', 'experiment_name': 'batch-06301354336591', 'compute': None, 'services': {'Tracking': {'endpoint': 'azureml://northeurope.api.azureml.ms/mlflow/v1.0/subscriptions/604342bf-c5b7-46e8-8eef-def8c102e8b6/resourceGroups/rg-dp100-l00b7cf28ae7f478cbf/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-l00b7cf28ae7f478cbf?', 'type': 'Tracking'}, 'Studio': {'endpoint': 'https://ml.azure.com/runs/batchjob-9b7f10a4-690d-43a4-8e74-22ac1c58dca2?wsid=/subscriptions/604342bf-c5b7-46e8-8eef-def8c102e8b6/resourcegroups/rg-dp100-l00b7cf28ae7f478cbf/workspaces/mlw-dp100-l00b7cf28ae7f478cbf&tid=4cfe372a-37a4-44f8-91b2-5faf34253c62', 'type': 'Studio'}}, 'settings': {}, 'identity': None, 'default_code': None, 'default_environment': None})

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [19]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

JobException: This job is in state Running. Download is allowed only in states ['Completed', 'Failed', 'Canceled', 'NotResponding', 'Paused']

In [ ]:
import pandas as pd

score = pd.read_csv("predictions.csv", index_col=0, names=["Date", "prediction", "file"])
score